In [1]:
import math
import os
import time
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

In [3]:
def load_data(path):
    data = pd.read_csv(path)
    data = data.drop(columns = ['Unnamed: 0'])
    data = data[1:]
    print(data.shape, (data == 0).sum().sum())
    return data

In [4]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
    return trainX, trainY, testX, testY

In [5]:
def generate_model(seq_len, node_num, pre_len):
    model = tf.keras.Sequential([
        tf.keras.layers.GRU(units=64, return_sequences=True, input_shape = [seq_len, node_num]),
        tf.keras.layers.GRU(units=64),
        tf.keras.layers.Dense(pre_len * node_num)
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [6]:
def train_model(model, train_datas, realY):
    trainX, trainY, testX, testY = train_datas[0], train_datas[1], train_datas[2], train_datas[3]
    print(trainX.shape, trainY.shape, testX.shape, testY.shape, realY.shape)
    
    model.fit(trainX, trainY, epochs=20, batch_size=32)
    y_pred = model.predict(testX)
    print(y_pred.shape)
    
    rmse = math.sqrt(mean_squared_error(realY, y_pred))
    mae = mean_absolute_error(realY, y_pred)
    mape = mean_absolute_percentage_error(realY, y_pred)
    f_norm = la.norm(realY - y_pred) / la.norm(realY)
    acc = 1-f_norm
    
    print(rmse, mae, mape, acc)

In [7]:
path_0 = '../Data/METR-LA/speed_la_0.csv'
path_5 = '../Data/METR-LA/speed_la_5.csv'
path_10 = '../Data/METR-LA/speed_la_10.csv'
path_20 = '../Data/METR-LA/speed_la_20.csv'

In [8]:
train_rate = 0.8
seq_len = 12

In [9]:
mainData = load_data(path_0)
data_0 = load_data(path_0)
data_5 = load_data(path_5)
data_10 = load_data(path_10)
data_20 = load_data(path_20)

(34271, 207) 0
(34271, 207) 0
(34271, 207) 342710
(34271, 207) 685420
(34271, 207) 1405111


# Pred = 15min

In [10]:
pre_len = 3

In [11]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

34271 207


(6840, 621)

### Unobserved nodes = 0%

In [12]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621)
(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621) (6840, 621)
Epoch 1/20
857/857 [==============================] - 8s 4ms/step - loss: 1502.9126 - mae: 35.5230
Epoch 2/20
857/857 [==============================] - 8s 9ms/step - loss: 204.2280 - mae: 12.2335
Epoch 3/20
857/857 [==============================] - 8s 9ms/step - loss: 131.4650 - mae: 7.6707
Epoch 4/20
857/857 [==============================] - 7s 8ms/step - loss: 130.7554 - mae: 7.3923
Epoch 5/20
857/857 [==============================] - 8s 10ms/step - loss: 130.7639 - mae: 7.3941
Epoch 6/20
857/857 [==============================] - 7s 8ms/step - loss: 130.7667 - mae: 7.3903
Epoch 7/20
857/857 [==============================] - 7s 8ms/step - loss: 130.7756 - mae: 7.3912
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7929 - mae: 7.3912
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7852 - mae: 7.

### Unobserved nodes = 5%

In [13]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621)
(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621) (6840, 621)
Epoch 1/20
857/857 [==============================] - 12s 10ms/step - loss: 1435.7461 - mae: 33.8716
Epoch 2/20
857/857 [==============================] - 8s 10ms/step - loss: 193.9535 - mae: 11.6234
Epoch 3/20
857/857 [==============================] - 7s 9ms/step - loss: 124.1986 - mae: 7.2441
Epoch 4/20
857/857 [==============================] - 8s 9ms/step - loss: 123.5611 - mae: 6.9846
Epoch 5/20
857/857 [==============================] - 9s 10ms/step - loss: 123.5675 - mae: 6.9777
Epoch 6/20
857/857 [==============================] - 8s 10ms/step - loss: 123.5718 - mae: 6.9772
Epoch 7/20
857/857 [==============================] - 7s 8ms/step - loss: 123.5817 - mae: 6.9860
Epoch 8/20
857/857 [==============================] - 8s 10ms/step - loss: 123.6003 - mae: 6.9801
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5963 - ma

### Unobserved nodes = 10%

In [14]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621)
(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621) (6840, 621)
Epoch 1/20
857/857 [==============================] - 10s 8ms/step - loss: 1350.5264 - mae: 31.9818
Epoch 2/20
857/857 [==============================] - 9s 10ms/step - loss: 182.8601 - mae: 10.9837
Epoch 3/20
857/857 [==============================] - 9s 11ms/step - loss: 118.1636 - mae: 6.8881
Epoch 4/20
857/857 [==============================] - 8s 9ms/step - loss: 117.5265 - mae: 6.6508
Epoch 5/20
857/857 [==============================] - 8s 9ms/step - loss: 117.5289 - mae: 6.6455
Epoch 6/20
857/857 [==============================] - 8s 10ms/step - loss: 117.5454 - mae: 6.6375
Epoch 7/20
857/857 [==============================] - 8s 10ms/step - loss: 117.5486 - mae: 6.6422
Epoch 8/20
857/857 [==============================] - 7s 8ms/step - loss: 117.5575 - mae: 6.6447
Epoch 9/20
857/857 [==============================] - 9s 10ms/step - loss: 117.5579 - ma

### Unobserved nodes = 20%

In [15]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621)
(27401, 12, 207) (27401, 621) (6840, 12, 207) (6840, 621) (6840, 621)
Epoch 1/20
857/857 [==============================] - 6s 4ms/step - loss: 1187.0391 - mae: 28.2146
Epoch 2/20
857/857 [==============================] - 5s 6ms/step - loss: 159.5265 - mae: 9.6742
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 102.9210 - mae: 6.1258
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 102.3839 - mae: 5.9183
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 102.3878 - mae: 5.9093
Epoch 6/20
857/857 [==============================] - 6s 8ms/step - loss: 102.3985 - mae: 5.9126
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 102.4028 - mae: 5.9138
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 102.4064 - mae: 5.9103
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 102.4094 - mae: 5.90

# Pred = 30min

In [16]:
pre_len = 6

In [17]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

34271 207


(6837, 1242)

### Unobserved nodes = 0%

In [18]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242)
(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242) (6837, 1242)
Epoch 1/20
857/857 [==============================] - 9s 7ms/step - loss: 1502.1238 - mae: 35.5039
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 203.8766 - mae: 12.2177
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 131.4487 - mae: 7.6624
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7713 - mae: 7.4036
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7667 - mae: 7.3923
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7788 - mae: 7.3949
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7846 - mae: 7.3888
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7965 - mae: 7.3914A: 0s - loss: 130.6722 - mae:
Epoch 9/20
857/857 [==============================] - 5s 6m

### Unobserved nodes = 5%

In [19]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242)
(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242) (6837, 1242)
Epoch 1/20
857/857 [==============================] - 8s 7ms/step - loss: 1437.0582 - mae: 33.8902
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 193.9685 - mae: 11.6239
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 124.2315 - mae: 7.2457
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5738 - mae: 6.9899
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5758 - mae: 6.9794
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5851 - mae: 6.9793
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5898 - mae: 6.9789
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5967 - mae: 6.9794
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 123.6043 - mae

### Unobserved nodes = 10%

In [20]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242)
(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242) (6837, 1242)
Epoch 1/20
857/857 [==============================] - 6s 5ms/step - loss: 1362.6788 - mae: 32.1521
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 185.1276 - mae: 11.0831
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 118.1715 - mae: 6.9062
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5309 - mae: 6.6368
Epoch 5/20
857/857 [==============================] - 6s 8ms/step - loss: 117.5432 - mae: 6.6422
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5460 - mae: 6.6483
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5574 - mae: 6.6390
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5643 - mae: 6.6439
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5535 - mae

### Unobserved nodes = 20%

In [21]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242)
(27398, 12, 207) (27398, 1242) (6837, 12, 207) (6837, 1242) (6837, 1242)
Epoch 1/20
857/857 [==============================] - 9s 7ms/step - loss: 1190.3441 - mae: 28.2675
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 159.9064 - mae: 9.6934
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 102.9237 - mae: 6.1285
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 102.3918 - mae: 5.9151
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 102.3939 - mae: 5.9061
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 102.4014 - mae: 5.9146
Epoch 7/20
857/857 [==============================] - 5s 6ms/step - loss: 102.4092 - mae: 5.9120
Epoch 8/20
857/857 [==============================] - 3s 4ms/step - loss: 102.4188 - mae: 5.9141
Epoch 9/20
857/857 [==============================] - 3s 3ms/step - loss: 102.4250 - mae:

# Pred = 45min

In [22]:
pre_len = 9

In [23]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

34271 207


(6834, 1863)

### Unobserved nodes = 0%

In [24]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863)
(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863) (6834, 1863)
Epoch 1/20
857/857 [==============================] - 9s 7ms/step - loss: 1511.0881 - mae: 35.6267
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 205.8959 - mae: 12.2975
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 131.4950 - mae: 7.6831
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7778 - mae: 7.4001
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7813 - mae: 7.3920
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 130.7902 - mae: 7.3947
Epoch 7/20
857/857 [==============================] - 7s 8ms/step - loss: 130.8046 - mae: 7.3859
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 130.8013 - mae: 7.3960
Epoch 9/20
857/857 [==============================] - 6s 7ms/step - loss: 130.8123 - mae

### Unobserved nodes = 5%

In [25]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863)
(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863) (6834, 1863)
Epoch 1/20
857/857 [==============================] - 6s 4ms/step - loss: 1438.9631 - mae: 33.9240
Epoch 2/20
857/857 [==============================] - 3s 4ms/step - loss: 194.7126 - mae: 11.6533
Epoch 3/20
857/857 [==============================] - 3s 4ms/step - loss: 124.2894 - mae: 7.2565
Epoch 4/20
857/857 [==============================] - 4s 5ms/step - loss: 123.5799 - mae: 6.9882
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 123.5812 - mae: 6.9788
Epoch 6/20
857/857 [==============================] - ETA: 0s - loss: 123.5883 - mae: 6.982 - 6s 7ms/step - loss: 123.5767 - mae: 6.9821
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 123.6011 - mae: 6.9806
Epoch 8/20
857/857 [==============================] - 7s 8ms/step - loss: 123.6097 - mae: 6.9824
Epoch 9/20
857/857 [============================

### Unobserved nodes = 10%

In [26]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863)
(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863) (6834, 1863)
Epoch 1/20
857/857 [==============================] - 8s 7ms/step - loss: 1359.5983 - mae: 32.1036
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 184.3209 - mae: 11.0475
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 118.1825 - mae: 6.9014
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5429 - mae: 6.6495
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5395 - mae: 6.6340
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5505 - mae: 6.6498
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5588 - mae: 6.6431
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 117.5701 - mae: 6.6318
Epoch 9/20
857/857 [==============================] - 5s 6ms/step - loss: 117.5796 - mae

### Unobserved nodes = 20%

In [27]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863)
(27395, 12, 207) (27395, 1863) (6834, 12, 207) (6834, 1863) (6834, 1863)
Epoch 1/20
857/857 [==============================] - 9s 8ms/step - loss: 1190.6027 - mae: 28.2769
Epoch 2/20
857/857 [==============================] - 6s 7ms/step - loss: 159.9947 - mae: 9.6991
Epoch 3/20
857/857 [==============================] - 6s 7ms/step - loss: 102.9151 - mae: 6.1302
Epoch 4/20
857/857 [==============================] - 6s 7ms/step - loss: 102.3985 - mae: 5.9061
Epoch 5/20
857/857 [==============================] - 6s 7ms/step - loss: 98.2290 - mae: 5.9024
Epoch 6/20
857/857 [==============================] - 6s 7ms/step - loss: 86.2564 - mae: 5.5670
Epoch 7/20
857/857 [==============================] - 6s 7ms/step - loss: 81.1425 - mae: 5.2592
Epoch 8/20
857/857 [==============================] - 6s 7ms/step - loss: 78.6843 - mae: 5.0730
Epoch 9/20
857/857 [==============================] - 7s 8ms/step - loss: 78.3690 - mae: 5.01

# Pred = 60min

In [28]:
pre_len = 12

In [29]:
time_len = mainData.shape[0]
node_num = mainData.shape[1]

print(time_len, node_num)

_, _, _, realY = preprocess_data(mainData, time_len, train_rate, seq_len, pre_len)
realY = realY.reshape([-1, node_num * pre_len])
realY.shape

34271 207


(6831, 2484)

### Unobserved nodes = 0%

In [30]:
time_len = data_0.shape[0]
node_num = data_0.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_0, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484)
(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484) (6831, 2484)
Epoch 1/20
856/856 [==============================] - 5s 3ms/step - loss: 1501.9988 - mae: 35.5172
Epoch 2/20
856/856 [==============================] - 3s 3ms/step - loss: 204.7742 - mae: 12.2560
Epoch 3/20
856/856 [==============================] - 3s 4ms/step - loss: 131.4792 - mae: 7.6743
Epoch 4/20
856/856 [==============================] - 4s 5ms/step - loss: 130.7801 - mae: 7.4042
Epoch 5/20
856/856 [==============================] - 5s 6ms/step - loss: 130.7873 - mae: 7.3909
Epoch 6/20
856/856 [==============================] - 5s 6ms/step - loss: 130.8036 - mae: 7.3879
Epoch 7/20
856/856 [==============================] - 5s 6ms/step - loss: 130.8042 - mae: 7.3935
Epoch 8/20
856/856 [==============================] - 5s 6ms/step - loss: 130.8219 - mae: 7.3928
Epoch 9/20
856/856 [==============================] - 5s 6ms/step - loss: 130.8107 - mae

### Unobserved nodes = 5%

In [31]:
time_len = data_5.shape[0]
node_num = data_5.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_5, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484)
(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484) (6831, 2484)
Epoch 1/20
856/856 [==============================] - 7s 6ms/step - loss: 1437.8763 - mae: 33.9095
Epoch 2/20
856/856 [==============================] - 5s 6ms/step - loss: 194.7819 - mae: 11.6581
Epoch 3/20
856/856 [==============================] - 5s 6ms/step - loss: 124.2644 - mae: 7.2593
Epoch 4/20
856/856 [==============================] - 5s 6ms/step - loss: 123.5869 - mae: 6.9774
Epoch 5/20
856/856 [==============================] - 5s 6ms/step - loss: 123.5898 - mae: 6.9855
Epoch 6/20
856/856 [==============================] - 5s 6ms/step - loss: 123.5922 - mae: 6.9851
Epoch 7/20
856/856 [==============================] - 5s 6ms/step - loss: 123.6111 - mae: 6.9741
Epoch 8/20
856/856 [==============================] - 5s 6ms/step - loss: 123.6158 - mae: 6.9852
Epoch 9/20
856/856 [==============================] - 5s 6ms/step - loss: 123.6263 - mae

### Unobserved nodes = 10%

In [32]:
time_len = data_10.shape[0]
node_num = data_10.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_10, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484)
(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484) (6831, 2484)
Epoch 1/20
856/856 [==============================] - 7s 6ms/step - loss: 1362.0382 - mae: 32.1466
Epoch 2/20
856/856 [==============================] - 5s 6ms/step - loss: 185.2062 - mae: 11.0819
Epoch 3/20
856/856 [==============================] - 5s 6ms/step - loss: 118.1999 - mae: 6.9047
Epoch 4/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5472 - mae: 6.6468
Epoch 5/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5530 - mae: 6.6406
Epoch 6/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5615 - mae: 6.6392
Epoch 7/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5627 - mae: 6.6454
Epoch 8/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5701 - mae: 6.6440
Epoch 9/20
856/856 [==============================] - 5s 6ms/step - loss: 117.5819 - mae

### Unobserved nodes = 20%

In [33]:
time_len = data_20.shape[0]
node_num = data_20.shape[1]

trainX, trainY, testX, testY = preprocess_data(data_20, time_len, train_rate, seq_len, pre_len)
trainY = trainY.reshape([-1, pre_len * node_num])
testY = testY.reshape([-1, pre_len * node_num])
print(trainX.shape, trainY.shape, testX.shape, testY.shape)

model = generate_model(seq_len, node_num, pre_len)
train_model(model, (trainX,trainY,testX,testY), realY)

(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484)
(27392, 12, 207) (27392, 2484) (6831, 12, 207) (6831, 2484) (6831, 2484)
Epoch 1/20
856/856 [==============================] - 7s 6ms/step - loss: 1184.5098 - mae: 28.1892
Epoch 2/20
856/856 [==============================] - 5s 6ms/step - loss: 159.1259 - mae: 9.6632
Epoch 3/20
856/856 [==============================] - 5s 6ms/step - loss: 102.9046 - mae: 6.1203
Epoch 4/20
856/856 [==============================] - 5s 6ms/step - loss: 102.4040 - mae: 5.9156
Epoch 5/20
856/856 [==============================] - 5s 6ms/step - loss: 102.4081 - mae: 5.9130
Epoch 6/20
856/856 [==============================] - 4s 4ms/step - loss: 102.4146 - mae: 5.9056
Epoch 7/20
856/856 [==============================] - 3s 3ms/step - loss: 102.4211 - mae: 5.9193
Epoch 8/20
856/856 [==============================] - 3s 3ms/step - loss: 102.4292 - mae: 5.9117
Epoch 9/20
856/856 [==============================] - 3s 3ms/step - loss: 102.4340 - mae: